<a href="https://colab.research.google.com/github/kaganseyda/projects/blob/Machine-Learning-Trained-Models/ChatBot%20Prediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:

# IMPORTANT: RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES
# TO THE CORRECT LOCATION (/kaggle/input) IN YOUR NOTEBOOK,
# THEN FEEL FREE TO DELETE THIS CELL.
# NOTE: THIS NOTEBOOK ENVIRONMENT DIFFERS FROM KAGGLE'S PYTHON
# ENVIRONMENT SO THERE MAY BE MISSING LIBRARIES USED BY YOUR
# NOTEBOOK.

import os
import sys
from tempfile import NamedTemporaryFile
from urllib.request import urlopen
from urllib.parse import unquote, urlparse
from urllib.error import HTTPError
from zipfile import ZipFile
import tarfile
import shutil

CHUNK_SIZE = 40960
DATA_SOURCE_MAPPING = 'lmsys-chatbot-arena:https%3A%2F%2Fstorage.googleapis.com%2Fkaggle-competitions-data%2Fkaggle-v2%2F66631%2F8346466%2Fbundle%2Farchive.zip%3FX-Goog-Algorithm%3DGOOG4-RSA-SHA256%26X-Goog-Credential%3Dgcp-kaggle-com%2540kaggle-161607.iam.gserviceaccount.com%252F20240708%252Fauto%252Fstorage%252Fgoog4_request%26X-Goog-Date%3D20240708T172854Z%26X-Goog-Expires%3D259200%26X-Goog-SignedHeaders%3Dhost%26X-Goog-Signature%3D08638f47adc3aa5993efc57a24411d77254a50b67bd1a9f44ae6c4351efeb796880a25576d555d10961d836cac3ff42a696c487dde014083938763b14f31238aa4345b7c33c596ea0f3e521f58134c5521568958426f46d97057a5a5f9f1afaef169690f9472c2269c0e5e8734a57dcd1f9d9b24e73631adb9888ef17b0ca63201a0c1771cd3a3c5af36b62688abcc2f0019233b326e1dd05e3b87fd7aa3c938bd408cb7858a8a540b8e471f670e14fc0b56c8a8c54c02a9d2560c45363811443ab4e9381c5c8840f769e2a2dc25450503839c421bf5dc8d869915589b5dd3d0344e2fb109384f13e29f5d95b4f8f6b66589daa54a927c9e599cf095423fec3b'

KAGGLE_INPUT_PATH='/kaggle/input'
KAGGLE_WORKING_PATH='/kaggle/working'
KAGGLE_SYMLINK='kaggle'

!umount /kaggle/input/ 2> /dev/null
shutil.rmtree('/kaggle/input', ignore_errors=True)
os.makedirs(KAGGLE_INPUT_PATH, 0o777, exist_ok=True)
os.makedirs(KAGGLE_WORKING_PATH, 0o777, exist_ok=True)

try:
  os.symlink(KAGGLE_INPUT_PATH, os.path.join("..", 'input'), target_is_directory=True)
except FileExistsError:
  pass
try:
  os.symlink(KAGGLE_WORKING_PATH, os.path.join("..", 'working'), target_is_directory=True)
except FileExistsError:
  pass

for data_source_mapping in DATA_SOURCE_MAPPING.split(','):
    directory, download_url_encoded = data_source_mapping.split(':')
    download_url = unquote(download_url_encoded)
    filename = urlparse(download_url).path
    destination_path = os.path.join(KAGGLE_INPUT_PATH, directory)
    try:
        with urlopen(download_url) as fileres, NamedTemporaryFile() as tfile:
            total_length = fileres.headers['content-length']
            print(f'Downloading {directory}, {total_length} bytes compressed')
            dl = 0
            data = fileres.read(CHUNK_SIZE)
            while len(data) > 0:
                dl += len(data)
                tfile.write(data)
                done = int(50 * dl / int(total_length))
                sys.stdout.write(f"\r[{'=' * done}{' ' * (50-done)}] {dl} bytes downloaded")
                sys.stdout.flush()
                data = fileres.read(CHUNK_SIZE)
            if filename.endswith('.zip'):
              with ZipFile(tfile) as zfile:
                zfile.extractall(destination_path)
            else:
              with tarfile.open(tfile.name) as tarfile:
                tarfile.extractall(destination_path)
            print(f'\nDownloaded and uncompressed: {directory}')
    except HTTPError as e:
        print(f'Failed to load (likely expired) {download_url} to path {destination_path}')
        continue
    except OSError as e:
        print(f'Failed to load {download_url} to path {destination_path}')
        continue

print('Data source import complete.')


In [ ]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score





In [ ]:

import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.model_selection import RandomizedSearchCV
from scipy.stats import uniform

# Load data
train=pd.read_csv('/kaggle/input/lmsys-chatbot-arena/train.csv')

test=pd.read_csv('/kaggle/input/lmsys-chatbot-arena/test.csv')

In [ ]:
train=train.drop('model_b',axis=1)

train.head()

,id,response_a,response_b,winner_model_a,winner_model_b,winner_tie
0,30192,"[""The question of whether it is morally right ...","[""As an AI, I don't have personal beliefs or o...",1,0,0
1,53567,"[""A marriage license is a legal document that ...","[""A marriage license and a marriage certificat...",0,1,0
2,65089,"[""Function calling is the process of invoking ...","[""Function calling is the process of invoking ...",0,0,1
3,96401,"[""Creating a test set for a very rare category...","[""When building a classifier for a very rare c...",1,0,0
4,198779,"[""The best way to travel from Tel Aviv to Jeru...","[""The best way to travel from Tel-Aviv to Jeru...",0,1,0


In [ ]:


stopwords = ['i', 'me', 'my', 'myself', 'we', 'our', 'ours',
'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself',
'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself',
'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves',
 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are',
 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing',
 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for',
'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to',
'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once',
'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most',
'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', 'too', 'very',
's', 't', 'can', 'will', 'just', 'don', "don't", 'should', "should've", 'now', 'd', 'll', 'm', 'o', 're', 've', 'y',
 'ain', 'aren', "aren't", 'couldn', "couldn't", 'didn', "didn't", 'doesn', "doesn't",
'hadn', "hadn't", 'hasn', "hasn't", 'haven', "haven't", 'isn', "isn't", 'ma', 'mightn',
 "mightn't", 'mustn', "mustn't", 'needn', "needn't", 'shan', "shan't", 'shouldn', "shouldn't",
'wasn', "wasn't", 'weren', "weren't", 'won', "won't", 'wouldn', "wouldn't"]

for word in stopwords:
    word = " " + word + " "

    train['response_a'] = train['response_a'].str.replace(word, " ")
    train['response_b'] = train['response_b'].str.replace(word, " ")

    test['response_a'] = test['response_a'].str.replace(word, " ")
    test['response_b'] = test['response_b'].str.replace(word, " ")




In [ ]:
tfidf = TfidfVectorizer()
combined_text_train = train['response_a'] + ' ' + train['response_b']
combined_text_test =  test['response_a'] + ' ' + test['response_b']
tfidf.fit(combined_text_train)
X_train = tfidf.transform(combined_text_train)
X_test = tfidf.transform(combined_text_test)




In [ ]:
y_train = train[['winner_model_a', 'winner_model_b', 'winner_tie']]


mlp_model = MLPClassifier(hidden_layer_sizes=(250,), activation='logistic', solver='lbfgs',max_iter=1000, random_state=42)


mlp_model.fit(X_train, y_train)
pred_proba = mlp_model.predict_proba(X_test)


In [ ]:
accuracy = accuracy_score(X_test, pred_proba)



In [ ]:

pred_model_a_proba = pred_proba[:, 0]
pred_model_b_proba = pred_proba[:, 1]
pred_model_tie_proba = pred_proba[:, 2]


total_percentage = pred_model_a_proba + pred_model_b_proba + pred_model_tie_proba
pred_model_a_proba /= total_percentage
pred_model_b_proba /= total_percentage
pred_model_tie_proba /= total_percentage


submission_df = pd.DataFrame({
    'id': test['id'],
    'winner_model_a': pred_model_a_proba,
    'winner_model_b': pred_model_b_proba,
    'winner_tie': pred_model_tie_proba
})
submission_df.to_csv('submission.csv', index=False)